# ⚡ PromptForge: Local Colab Edition

Run the full PromptForge optimization pipeline **locally** on this Google Colab instance using a GPU-accelerated Llama.cpp backend.

**No API Keys required. No Cloud data transfer. 100% Private.**

### Selected Model:
By default, we will download and use **Phi-3 Mini (3.8B)** (Quantized) which fits perfectly on the free T4 GPU and provides excellent instruction adherence.

In [ ]:
# @title 1. Setup Environment (Fast Install)
# Check GPU
!nvidia-smi

# Install llama-cpp-python using pre-built CUDA wheels (Much faster than compiling)
!pip install llama-cpp-python --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cu121

# Install other dependencies
!pip install sqlalchemy pydantic ipywidgets huggingface_hub ollama openai anthropic google-generativeai groq

# Clone App
!git clone https://github.com/ahmadzkn/prompt-forge.git
%cd prompt-forge

# Fix for Colab environment import paths
import sys
import os
sys.path.append(os.getcwd())

In [ ]:
# @title 2. Download Model (Phi-3 Mini Instruct)
from huggingface_hub import hf_hub_download

model_repo = "microsoft/Phi-3-mini-4k-instruct-gguf"
model_filename = "Phi-3-mini-4k-instruct-q4.gguf"

print(f"Downloading {model_filename} from {model_repo}...")
try:
    model_path = hf_hub_download(repo_id=model_repo, filename=model_filename)
    print(f"\n✅ Model downloaded to: {model_path}")
except Exception as e:
    print(f"Download Error: {e}")

In [ ]:
# @title 3. Run Optimizer
import ipywidgets as widgets
from IPython.display import display, clear_output
from src.optimizer import PromptOptimizer
import traceback

# Initialize with LlamaCpp
print("🚀 Loading Model into GPU memory... (This takes a few seconds)")

try:
    optimizer = PromptOptimizer(
        provider_type="llamacpp",
        model_path=model_path,
        n_gpu_layers=-1, # Offload all to GPU
        n_ctx=4096
    )
    print("✅ Model Loaded Successfully!")
except Exception as e:
    print(f"❌ Error loading model: {e}")
    traceback.print_exc()

# UI
raw_prompt_area = widgets.Textarea(
    placeholder='Enter your raw prompt here...',
    description='Raw Prompt:',
    layout=widgets.Layout(width='100%', height='150px')
)

optimize_btn = widgets.Button(
    description='Optimize Prompt',
    button_style='primary',
    icon='magic'
)

output_area = widgets.Output()

def on_optimize(b):
    with output_area:
        clear_output()
        print("Optimizing... Please wait.")
        try:
            # 'local' model name is ignored by LlamaCpp backend but required by interface
            result = optimizer.optimize_prompt(raw_prompt_area.value, model="local")
            
            if "error" in result:
                print(f"❌ Error: {result['error']}")
                return

            print("### Final Optimized Prompt")
            print("-" * 80)
            print(result.get("final_prompt", "No result"))
            print("-" * 80)
            
            print("\n### Structured Elements:")
            elements = result.get("elements", {})
            for k, v in elements.items():
                print(f"**{k.title()}**: {v}")
        except Exception as e:
            print(f"❌ Execution Error: {e}")
            traceback.print_exc()

optimize_btn.on_click(on_optimize)

display(widgets.VBox([
    widgets.HTML("<h2>PromptForge Local Optimizer</h2>"),
    widgets.HTML("<p>Enter a raw idea and let the local Phi-3 model optimize it for you.</p>"),
    raw_prompt_area,
    optimize_btn,
    output_area
]))